In [22]:
import csv
import re
import numpy as np

param_re = re.compile(r"ch(\d+)=(\-?\d+\.?\d*e?[+\-]?\d*)", re.IGNORECASE)

def parse_cadence_csv(path):
    """
    Returns:
      challenges: np.ndarray (256, 8) with voltages (ch0..ch7)
      diffs:      np.ndarray (256,) with diff values
      bits:       np.ndarray (256,) with 0/1 (1 if diff>0)
    """
    challenges = []
    diffs = []

    with open(path, newline='') as f:
        reader = csv.reader(f)
        current_ch = None
        for row in reader:
            if not row:
                continue
            # Parameters row
            if row[0].startswith("Parameters:"):
                line = row[0]
                vals = {int(m.group(1)): float(m.group(2))
                        for m in param_re.finditer(line)}
                current_ch = [vals[i] for i in range(8)]  # ch0..ch7
            # Data row
            elif row[1:3] == ['base_setup', 'diff'] and row[3] != '':
                diff = float(row[3])
                if current_ch is None:
                    continue
                challenges.append(current_ch)
                diffs.append(diff)

    challenges = np.array(challenges)          # (N,8)
    diffs = np.array(diffs)                    # (N,)
    bits = (diffs > 0).astype(int)
    return challenges, diffs, bits

challenges, diffs, outputs = parse_cadence_csv("data/onlydiff.csv")


In [23]:
print(challenges.shape)  
print(outputs.shape)

(256, 8)
(256,)


In [24]:
import numpy as np

# challenges : (256, 8)
# outputs    : (256,)

uniformity = outputs.mean()
print("Uniformity (fraction of 1s):", uniformity)

Uniformity (fraction of 1s): 0.4921875


In [1]:
import csv
import re
import numpy as np

param_re = re.compile(r"ch(\d+)=(\-?\d+\.?\d*e?[+\-]?\d*)", re.IGNORECASE)
mc_re    = re.compile(r"mc_iteration=(\d+)")

def parse_two_chips(path):
    """
    path: CSV exported from ADE (your paste.txt)
    returns:
      challenges      : (N, 8) float (same for both chips)
      outputs_chip1   : (N,) int {0,1}
      outputs_chip2   : (N,) int {0,1}
    """
    # temporary dict: key = (tuple(ch0..ch7)), value = {1: diff, 2: diff}
    data = {}

    with open(path, newline='') as f:
        reader = csv.reader(f)
        current_ch = None
        current_mc = None

        for row in reader:
            if not row:
                continue

            # Parameters line
            if row[0].startswith("Parameters:"):
                line = row[0]

                # ch0..ch7
                vals = {int(m.group(1)): float(m.group(2))
                        for m in param_re.finditer(line)}
                current_ch = tuple(vals[i] for i in range(8))

                # mc_iteration
                m = mc_re.search(line)
                current_mc = int(m.group(1)) if m else None

            # Data line with diff
            elif row[1:3] == ['base_setup', 'diff'] and row[3] != '' and current_ch is not None:
                diff = float(row[3])
                if current_ch not in data:
                    data[current_ch] = {}
                if current_mc is not None:
                    data[current_ch][current_mc] = diff

    # Sort challenges in a stable order
    chal_list = sorted(data.keys())
    challenges = np.array(chal_list, dtype=float)

    # Build outputs per chip
    diffs1 = []
    diffs2 = []
    for ch in chal_list:
        diffs1.append(data[ch].get(1, np.nan))
        diffs2.append(data[ch].get(2, np.nan))

    diffs1 = np.array(diffs1)
    diffs2 = np.array(diffs2)

    outputs_chip1 = (diffs1 > 0).astype(int)
    outputs_chip2 = (diffs2 > 0).astype(int)

    return challenges, outputs_chip1, outputs_chip2


# ---------- example run ----------

path = "data/MonteCarlo.2.csv"
challenges, out1, out2 = parse_two_chips(path)
print("challenges shape:", challenges.shape)   # (256, 8)
print("chip1 outputs :", out1.shape)
print("chip2 outputs :", out2.shape)


challenges shape: (256, 8)
chip1 outputs : (256,)
chip2 outputs : (256,)


In [2]:
import numpy as np

# already have:
# challenges : (256, 8)
# out1       : (256,)
# out2       : (256,)

# stack outputs to get (n_chips, n_challenges)
all_outputs = np.stack([out1, out2], axis=0)   # shape (2, 256)


def uniformity(outputs_chip):
    """fraction of 1s for a single chip"""
    return outputs_chip.mean()


def inter_chip_uniqueness(outputs):
    """
    outputs: (n_chips, n_challenges)
    returns (mean_HD, std_HD, all_pair_HDs_array)
    """
    n_chips, n_ch = outputs.shape
    dists = []
    for i in range(n_chips - 1):
        xor = outputs[i] ^ outputs[i + 1:]
        hd = xor.mean(axis=1)
        dists.append(hd)
    if not dists:
        return np.nan, np.nan, np.array([])
    dists = np.concatenate(dists)
    return dists.mean(), dists.std(), dists


def bit_aliasing(outputs):
    """
    outputs: (n_chips, n_challenges)
    """
    return outputs.mean(axis=0)


# ----- run metrics -----

# Uniformity
u1 = uniformity(out1)
u2 = uniformity(out2)
print("Uniformity chip1:", u1)
print("Uniformity chip2:", u2)
print("Uniformity mean over chips:", (u1 + u2) / 2)

# Inter‑chip HD / uniqueness
uniq_mean, uniq_std, uniq_all = inter_chip_uniqueness(all_outputs)
print("Inter‑chip HD mean (uniqueness):", uniq_mean)
print("Inter‑chip HD std :", uniq_std)

# Bit‑aliasing (columns of uniqueness matrix)
alias = bit_aliasing(all_outputs)          # shape (256,)
print("Bit‑aliasing mean over 256 indices:", alias.mean())
print("Bit‑aliasing std over 256 indices :", alias.std())

# For sanity: HD between the two chips directly
hd_1_2 = (out1 != out2).mean()
print("Direct HD(out1,out2):", hd_1_2)


Uniformity chip1: 0.5703125
Uniformity chip2: 0.55859375
Uniformity mean over chips: 0.564453125
Inter‑chip HD mean (uniqueness): 0.12109375
Inter‑chip HD std : 0.0
Bit‑aliasing mean over 256 indices: 0.564453125
Bit‑aliasing std over 256 indices : 0.4642977031794734
Direct HD(out1,out2): 0.12109375
